In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import pandas as pd

df = pd.read_csv("final_dataset.csv")
df = df[['text','label']]


In [ ]:
df = df.sample(2000, random_state=42).reset_index(drop=True)


In [ ]:
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'], df['label'], test_size=0.2, random_state=42
)


In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)


In [ ]:
import torch

class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(train_encodings, list(train_labels))
test_dataset = NewsDataset(test_encodings, list(test_labels))


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./roberta_model',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_steps=20,
    save_steps=100,
    report_to="none"   # no wandb / tensorboard
)



In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

trainer.train()


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
20,0.352700
40,0.000800
60,0.095000
80,0.003400
100,0.097300
120,0.000200
140,0.000100
160,0.067900
180,0.000200
200,0.000200


TrainOutput(global_step=200, training_loss=0.061767058252589775, metrics={'train_runtime': 9693.7311, 'train_samples_per_second': 0.165, 'train_steps_per_second': 0.021, 'total_flos': 420977688576000.0, 'train_loss': 0.061767058252589775, 'epoch': 1.0})

In [ ]:
trainer.save_model("./roberta_model")
tokenizer.save_pretrained("./roberta_model")


('roberta_final/tokenizer_config.json',
 'roberta_final/special_tokens_map.json',
 'roberta_final/vocab.json',
 'roberta_final/merges.txt',
 'roberta_final/added_tokens.json')

In [ ]:
!zip -r roberta_model.zip roberta_model


  adding: roberta_model/ (stored 0%)
  adding: roberta_model/checkpoint-100/ (stored 0%)
  adding: roberta_model/checkpoint-100/training_args.bin (deflated 53%)
  adding: roberta_model/checkpoint-100/model.safetensors (deflated 13%)
  adding: roberta_model/checkpoint-100/optimizer.pt (deflated 24%)
  adding: roberta_model/checkpoint-100/scheduler.pt (deflated 62%)
  adding: roberta_model/checkpoint-100/rng_state.pth (deflated 26%)
  adding: roberta_model/checkpoint-100/config.json (deflated 50%)
  adding: roberta_model/checkpoint-100/trainer_state.json (deflated 65%)
  adding: roberta_model/checkpoint-200/ (stored 0%)
  adding: roberta_model/checkpoint-200/training_args.bin (deflated 53%)
  adding: roberta_model/checkpoint-200/model.safetensors (deflated 11%)
  adding: roberta_model/checkpoint-200/optimizer.pt (deflated 21%)
  adding: roberta_model/checkpoint-200/scheduler.pt (deflated 62%)
  adding: roberta_model/checkpoint-200/rng_state.pth (deflated 26%)
  adding: roberta_model/chec

In [ ]:
from google.colab import files
files.download("roberta_model.zip")
